# Testing the non-redundant sampling algorithm

In [ ]:
import infrared as ir
import itertools
import random
import string

def generate_random_graph(num_nodes):
    all_cycles = list(itertools.combinations(range(1, num_nodes + 1), 4))
    num_cycles = random.randint(1, len(all_cycles))
    cycles = random.sample(all_cycles, num_cycles)
    
    edges = set()
    for cycle in cycles:
        edges.add(tuple(sorted((cycle[0], cycle[1]))))
        edges.add(tuple(sorted((cycle[1], cycle[2]))))
        edges.add(tuple(sorted((cycle[2], cycle[3]))))
        edges.add(tuple(sorted((cycle[3], cycle[0]))))

    return list(edges), cycles


def create_model(num_nodes, num_colors, edges, cycles, weight):
    model = ir.Model()

    model.add_variables(1,(0,0))
    model.add_variables(num_nodes, num_colors)

    ir.def_constraint_class('NotEquals', lambda i,j: [i,j], lambda x, y: x!=y)
    model.add_constraints(NotEquals(i,j) for i,j in edges)

    ir.def_function_class('Card', lambda i, j, k, l: [i,j,k,l], lambda x,y,z,w: len({x,y,z,w}))
    model.add_functions([Card(i,j,k,l) for i,j,k,l in cycles], 'card')
    model.set_feature_weight(weight,'card')

    return model


def assignment_to_coloring(a, num_colors):
    colors=list(string.ascii_lowercase[:num_colors])
    coloring = {i:colors[v] for i,v in enumerate(a.values())}
    del coloring[0]
    return coloring


def are_different(list_of_dicts):
    seen = set()
    for d in list_of_dicts:
        dict_tuple = tuple(sorted(d.items()))
        if dict_tuple in seen:
            return False
        seen.add(dict_tuple)
    return True

### Test 1

In [ ]:
def test_naive_sampling(solver, repeats, num_colors):
    samples = []
    for _ in range(repeats):
        assignment = solver.sample_new(non_redundant=True, non_redundant_mode="rejection", repeats=repeats)
        if assignment:
            coloring = assignment_to_coloring(assignment, num_colors)
            samples.append(coloring)
        else:
            break
    return samples

def test_non_redundant_sampling(solver, repeats, num_colors):
    samples = []
    for _ in range(repeats):
        assignment = solver.sample_new(non_redundant=True)
        if assignment:
            coloring = assignment_to_coloring(assignment, num_colors)
            samples.append(coloring)
        else:
            break
    return samples

In [ ]:
import time

i = 0
while i < 100:
    num_colors = random.randint(3, 5)
    num_nodes = random.randint(4, 8)
    edges,cycles = generate_random_graph(num_nodes)
    weight = random.randint(-2, 2)

    model = create_model(num_nodes, num_colors, edges, cycles, weight)
    solver = ir.Sampler(model)
    
    try:
        solver.evaluate()
        i += 1
    except:
        continue

    try:
        print(f'\nTest #{i}\n')
        print(f'Coloring a graph with {num_nodes} nodes in {num_colors} colors with weight {weight}')
        print(f'edges: {edges}')
        print(f'cycles: {cycles}\n')
        
        repeats = num_nodes ** num_colors

        print(f'Testing the naive sampling method')
        start_time = time.time()
        naive_samples = test_naive_sampling(solver, repeats, num_colors)
        end_time = time.time()
        print(f'{len(naive_samples)} samples found')
        assert(are_different(naive_samples))
        print(f'run time: {end_time-start_time} seconds\n')

        print(f'Testing the non-redundant sampling method')
        start_time = time.time()
        non_redundant_samples = test_non_redundant_sampling(solver, repeats, num_colors)
        end_time = time.time()
        print(f'{len(non_redundant_samples)} samples found')
        assert(are_different(non_redundant_samples))
        print(f'run time: {end_time-start_time} seconds\n')
    
    except:
        print('An error occured\n')

### Test 2

In [ ]:
def test_naive_sampling_full(solver, repeats, num_colors, num_solutions):
    samples = []
    start = time.time()
    while len(samples) < num_solutions:
        assignment = solver.sample_new(non_redundant=True, non_redundant_mode="rejection", repeats=repeats)
        if assignment:
            coloring = assignment_to_coloring(assignment, num_colors)
            samples.append(coloring)
        if time.time() - start > 120:
            print('The naive method is taking too long\n')
            break
    return samples

In [ ]:
i = 0
while i < 100:
    num_colors = random.randint(3, 5)
    num_nodes = random.randint(4, 8)
    edges,cycles = generate_random_graph(num_nodes)
    weight = random.randint(-2, 2)

    model = create_model(num_nodes, num_colors, edges, cycles, weight)
    solver = ir.Sampler(model)
    
    try:
        solver.evaluate()
        i += 1
    except:
        continue

    try:
        print(f'\nTest #{i}\n')
        print(f'Coloring a graph with {num_nodes} nodes in {num_colors} colors with weight {weight}')
        print(f'edges: {edges}')
        print(f'cycles: {cycles}\n')
        
        repeats = num_nodes ** num_colors

        start_time_non_redundant = time.time()
        non_redundant_samples = test_non_redundant_sampling(solver, repeats, num_colors)
        end_time_non_redundant = time.time()
        assert(are_different(non_redundant_samples))

        start_time_naive = time.time()
        naive_samples = test_naive_sampling_full(solver, repeats, num_colors, len(non_redundant_samples))
        end_time_naive = time.time()
        assert(are_different(naive_samples))

        print(f'Expected {len(non_redundant_samples)} samples')
        print(f'Naive method run time:         {end_time_naive-start_time_naive} seconds')
        print(f'Non-redundant method run time: {end_time_non_redundant-start_time_non_redundant} seconds\n')
    
    except:
        print('An error occured\n')

### Test 3

In [ ]:
def test_naive_sampling_step(solver):
    start = time.time()
    while True:
        assignment = solver.sample_new(non_redundant=True, non_redundant_mode="rejection")
        if assignment:
            break
        if time.time() - start > 120:
            print('\nThe naive method is taking too long')
            break

def test_non_redundant_sampling_step(solver):
    assignment = solver.sample_new(non_redundant=True)
    if assignment:
        return True

In [ ]:
import matplotlib.pyplot as plt

i = 0
while i < 100:
    num_colors = random.randint(3, 5)
    num_nodes = random.randint(4, 8)
    edges,cycles = generate_random_graph(num_nodes)
    weight = random.randint(-2, 2)

    model = create_model(num_nodes, num_colors, edges, cycles, weight)
    solver = ir.Sampler(model)
    
    try:
        solver.evaluate()
        i += 1
    except:
        continue

    try:
        print(f'\nTest #{i}\n')
        print(f'Coloring a graph with {num_nodes} nodes in {num_colors} colors with weight {weight}')
        print(f'edges: {edges}')
        print(f'cycles: {cycles}')

        naive_times = []
        non_redundant_times = []

        while True:
            start_time_non_redundant = time.time()
            proceed = test_non_redundant_sampling_step(solver)
            end_time_non_redundant = time.time()
            
            if not proceed:
                break
            non_redundant_times.append(end_time_non_redundant-start_time_non_redundant)
            
            start_time_naive = time.time()
            test_naive_sampling_step(solver)
            end_time_naive = time.time()
            
            naive_times.append(end_time_naive-start_time_naive)
        
        plt.plot(naive_times, label='naive method')
        plt.plot(non_redundant_times, label='non-redundant method')
        plt.xlabel('sample')
        plt.ylabel('time')
        plt.legend()
        plt.show()
    
    except:
        print('An error occured\n')